In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load preference data
preference_df = pd.read_csv("synthetic-dataset/preference.csv")
user_item_matrix = preference_df.pivot(index='user_id', columns='activity_id', values='preference').fillna(0)

# Adjust preference values
user_item_matrix[user_item_matrix == -1] = np.nan

user_item_matrix.head()


activity_id,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.857805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.326088,NaN,...,NaN,0.135851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025994,...,NaN,0.048914,NaN,0.397286,NaN,NaN,NaN,NaN,NaN,NaN
2,0.333203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.063177,NaN,NaN,NaN,...,0.478959,NaN,NaN,0.462059,NaN,NaN,NaN,0.123057,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# see how cosine similarity works
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
import numpy as np

a = np.array([0, 2, 4])
b = np.array([1, 4, 2])
a = a -np.mean(a).repeat(len(a))
b = b - np.mean(b).repeat(len(b))
a = a / np.linalg.norm(a)
b = b / np.linalg.norm(b)
mat = np.stack((a, b), axis=0)

print(spatial.distance.cosine(a, b))
print(cosine_similarity(mat))

0.6726731646460115
[[1.         0.32732684]
 [0.32732684 1.        ]]


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

# Load preference data
preference_df = pd.read_csv("synthetic-dataset/preference.csv")
user_item_matrix = preference_df.pivot(index='user_id', columns='activity_id', values='preference')

# Replace -1 with NaN to indicate missing data
user_item_matrix.replace(-1, np.nan, inplace=True)

# Fill NaN values with the mean of each user's preferences for the purpose of computing similarities
mean_user_preferences = user_item_matrix.mean(axis=1)
mean_user_preferences = mean_user_preferences.fillna(mean_user_preferences.mean())
user_item_matrix_filled = user_item_matrix.T.fillna(mean_user_preferences).T

# Compute cosine similarity between users
user_similarity_matrix = cosine_similarity(user_item_matrix_filled)

print("Dimensions of user similarity matrix:", user_similarity_matrix.shape)
print("Number of users:", user_similarity_matrix.shape[0])
print("Number of items:", user_item_matrix_filled.shape[1])
print("length of one row of user item matrix:", len(user_item_matrix_filled.iloc[0]))

# Function to predict preferences
def predict_preferences(user_id):
    # Weights are the similarities with other users
    weights = user_similarity_matrix[user_id]
    # Get the ratings from all users
    ratings = user_item_matrix_filled.values
    # Compute the weighted sum of ratings
    weighted_ratings = np.dot(weights, ratings)
    # Compute the sum of weights
    sum_of_weights = np.array([np.abs(weights).sum() for _ in range(ratings.shape[1])])
    # Predicted preferences
    predicted_preferences = weighted_ratings / sum_of_weights
    # Replace known values with original to not predict them
    predicted_preferences[user_item_matrix.iloc[user_id].notna().values] = user_item_matrix.iloc[user_id].values[user_item_matrix.iloc[user_id].notna().values]
    return predicted_preferences

# Example: predict preferences for user 0
predicted_preferences_user_0 = predict_preferences(0)
print("Predicted preferences for user 0:", predicted_preferences_user_0)


Dimensions of user similarity matrix: (50, 50)
Number of users: 50
Number of items: 112
length of one row of user item matrix: 112
Predicted preferences for user 0: [0.85780529 0.35253686 0.36487012 0.34057706 0.34965487 0.35748239
 0.35655006 0.36324236 0.32608791 0.33814326 0.37336179 0.01153075
 0.36733297 0.06551385 0.36985726 0.35062908 0.34786488 0.36521187
 0.36197124 0.35456622 0.36235535 0.40782286 0.34287701 0.35612343
 0.3719619  0.35211553 0.38840399 0.37667856 0.3795545  0.36608189
 0.37549445 0.39033968 0.34364356 0.36779758 0.36504381 0.45374467
 0.35000953 0.3873457  0.36239026 0.38163414 0.32780437 0.36888261
 0.36302857 0.36651608 0.3762667  0.20439558 0.37486773 0.40414797
 0.38833085 0.74587186 0.37206441 0.34699016 0.38239891 0.3780616
 0.36999007 0.37510389 0.35567761 0.37376285 0.38715368 0.34862159
 0.37181731 0.34352315 0.08619683 0.33324827 0.36019562 0.39603515
 0.35845526 0.36123902 0.36386465 0.36156704 0.37342843 0.11202992
 0.34680219 0.33093084 0.0328185

In [6]:
from sklearn.metrics import mean_squared_error

# Predict preferences for all user-item pairs
all_predictions = np.zeros_like(user_item_matrix.values)
for i in range(user_item_matrix.shape[0]):
    all_predictions[i, :] = predict_preferences(i)

# Calculate MSE
mse = mean_squared_error(user_item_matrix_filled.values.flatten(), all_predictions.flatten())
print("Mean Squared Error:", mse)


Mean Squared Error: 0.005588789684686886


### Make some analysis on the predictions

In [7]:
users = pd.read_csv("synthetic-dataset/user.csv")
activities = pd.read_csv("synthetic-dataset/activity.csv", sep='|')
user_id = np.random.choice(users['user_id'])
print("User ID:", user_id)

true_user_condition = users[users['user_id'] == user_id].values[0][1]
print("True user conditions:", true_user_condition)

predicted_preferences = predict_preferences(user_id)
print("Predicted preferences:", predicted_preferences)

# Find the top 5 activities with the highest predicted preferences
top_activities = np.argsort(predicted_preferences)[::-1][:5]
print("Top 5 activities:", top_activities)

# Get the target condition of the top activities
target_conditions = activities.loc[top_activities, 'target_condition'].values
print("Target conditions of top activities:", target_conditions)

User ID: 23
True user conditions: 2
Predicted preferences: [0.32546349 0.35223193 0.36422767 0.34046168 0.34961266 0.35737752
 0.35643047 0.36335189 0.21523013 0.24657942 0.37338116 0.15352099
 0.36748693 0.39159937 0.37036005 0.35073479 0.88446449 0.3652589
 0.36221723 0.35475488 0.36240015 0.40814893 0.34326431 0.35608694
 0.37143342 0.15563769 0.38784409 0.37590888 0.37936491 0.36612425
 0.37583371 0.39060027 0.34381723 0.13841965 0.36508296 0.372157
 0.35008934 0.38725211 0.36220797 0.73522808 0.3452232  0.36938318
 0.36304795 0.36682517 0.37648735 0.37196846 0.37524476 0.40412985
 0.38843209 0.38710635 0.37158265 0.20913413 0.34951243 0.3775062
 0.41606397 0.37545581 0.35576863 0.37385856 0.38737192 0.34900018
 0.37191177 0.34370014 0.409299   0.37627467 0.36043108 0.84714861
 0.35844261 0.36131996 0.36393226 0.36196144 0.37361129 0.3818023
 0.34666248 0.39280423 0.34852301 0.3554011  0.3670577  0.36551885
 0.37590534 0.36003009 0.34026003 0.33522348 0.3709208  0.39250664
 0.36766

In [8]:
# for each possible condition, we want to measure the accuracy of the predictions
# for users with that condition

def custom_accuracy(user, prediction):
    # Get the user's condition
    user_condition = users[users['user_id'] == user]['condition_id'].values[0]
    # Get the target condition of the top activities
    top_activities = np.argsort(prediction)[::-1][:5]
    target_conditions = activities.loc[top_activities, 'target_condition'].values
    # measure the accuracy by summing a coefficient for each suggestion, decreasing with the rank
    accuracy = 0
    for rk, condition in enumerate(target_conditions):
        if condition == user_condition:
            accuracy += 0.5 / 2**rk

    return accuracy

# Get the unique target conditions
unique_conditions = activities['target_condition'].unique()

# Initialize a dictionary to store the MSE for each condition
mse_by_condition = {condition: 0 for condition in unique_conditions}
accuracy_by_condition = {condition: 0 for condition in unique_conditions}

# Iterate over each condition
for condition in unique_conditions:
    # Get the user ids with the target condition
    user_ids = users[users['condition_id'] == condition]['user_id'].values
    # Calculate the MSE for the users with the target condition
    mse_by_condition[condition] = mean_squared_error(user_item_matrix_filled.loc[user_ids].values.flatten(), all_predictions[user_ids].flatten())
    # Calculate the accuracy for the users with the target condition
    for user_id in user_ids:
        accuracy_by_condition[condition] += custom_accuracy(user_id, all_predictions[user_id])
    # normalize the accuracy
    accuracy_by_condition[condition] /= len(user_ids)

print("MSE by condition:", mse_by_condition)
print("Accuracy by condition:", accuracy_by_condition)

MSE by condition: {0: 0.003075764508498545, 1: 0.011474258762200379, 2: 0.003934381511311087, 3: 0.005143273648765628}
Accuracy by condition: {0: 0.8459821428571429, 1: 0.96875, 2: 0.853125, 3: 0.751953125}


In [1]:
import pandas as pd
import numpy as np
import time
import psutil
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy import spatial
import matplotlib.pyplot as plt

# Function to monitor CPU usage
def monitor_cpu():
    cpu_percent = psutil.cpu_percent(interval=1)
    return cpu_percent

# Function to monitor memory usage
def monitor_memory():
    memory_info = psutil.virtual_memory()
    return memory_info.percent

# Lists to store performance data
cpu_usages = []
memory_usages = []
execution_times = []

# Measure start time and CPU usage
start_time = time.time()
cpu_usage_start = monitor_cpu()
memory_usage_start = monitor_memory()

# Load preference data
preference_df = pd.read_csv("synthetic-dataset/preference.csv")
user_item_matrix = preference_df.pivot(index='user_id', columns='activity_id', values='preference').fillna(0)

# Adjust preference values
user_item_matrix[user_item_matrix == -1] = np.nan

user_item_matrix.head()

# See how cosine similarity works
a = np.array([0, 2, 4])
b = np.array([1, 4, 2])
a = a - np.mean(a)
b = b - np.mean(b)
a = a / np.linalg.norm(a)
b = b / np.linalg.norm(b)
mat = np.stack((a, b), axis=0)

print(spatial.distance.cosine(a, b))
print(cosine_similarity(mat))

# Replace -1 with NaN to indicate missing data
user_item_matrix.replace(-1, np.nan, inplace=True)

# Fill NaN values with the mean of each user's preferences for the purpose of computing similarities
mean_user_preferences = user_item_matrix.mean(axis=1)
mean_user_preferences = mean_user_preferences.fillna(mean_user_preferences.mean())
user_item_matrix_filled = user_item_matrix.T.fillna(mean_user_preferences).T

# Compute cosine similarity between users
user_similarity_matrix = cosine_similarity(user_item_matrix_filled)

print("Dimensions of user similarity matrix:", user_similarity_matrix.shape)
print("Number of users:", user_similarity_matrix.shape[0])
print("Number of items:", user_item_matrix_filled.shape[1])
print("Length of one row of user item matrix:", len(user_item_matrix_filled.iloc[0]))

# Function to predict preferences
def predict_preferences(user_id):
    # Weights are the similarities with other users
    weights = user_similarity_matrix[user_id]
    # Get the ratings from all users
    ratings = user_item_matrix_filled.values
    # Compute the weighted sum of ratings
    weighted_ratings = np.dot(weights, ratings)
    # Compute the sum of weights
    sum_of_weights = np.array([np.abs(weights).sum() for _ in range(ratings.shape[1])])
    # Predicted preferences
    predicted_preferences = weighted_ratings / sum_of_weights
    # Replace known values with original to not predict them
    predicted_preferences[user_item_matrix.iloc[user_id].notna().values] = user_item_matrix.iloc[user_id].values[user_item_matrix.iloc[user_id].notna().values]
    return predicted_preferences

# Example: predict preferences for user 0
predicted_preferences_user_0 = predict_preferences(0)
print("Predicted preferences for user 0:", predicted_preferences_user_0)

# Predict preferences for all user-item pairs
all_predictions = np.zeros_like(user_item_matrix.values)
for i in range(user_item_matrix.shape[0]):
    all_predictions[i, :] = predict_preferences(i)
    # Capture performance metrics
    cpu_usages.append(monitor_cpu())
    memory_usages.append(monitor_memory())
    execution_times.append(time.time() - start_time)

# Calculate MSE
mse = mean_squared_error(user_item_matrix_filled.values.flatten(), all_predictions.flatten())
print("Mean Squared Error:", mse)

users = pd.read_csv("synthetic-dataset/user.csv")
activities = pd.read_csv("synthetic-dataset/activity.csv", sep='|')
user_id = np.random.choice(users['user_id'])
print("User ID:", user_id)

true_user_condition = users[users['user_id'] == user_id].values[0][1]
print("True user conditions:", true_user_condition)

predicted_preferences = predict_preferences(user_id)
print("Predicted preferences:", predicted_preferences)

# Find the top 5 activities with the highest predicted preferences
top_activities = np.argsort(predicted_preferences)[::-1][:5]
print("Top 5 activities:", top_activities)

# Get the target condition of the top activities
target_conditions = activities.loc[top_activities, 'target_condition'].values
print("Target conditions of top activities:", target_conditions)

# Function to measure custom accuracy
def custom_accuracy(user, prediction):
    # Get the user's condition
    user_condition = users[users['user_id'] == user]['condition_id'].values[0]
    # Get the target condition of the top activities
    top_activities = np.argsort(prediction)[::-1][:5]
    target_conditions = activities.loc[top_activities, 'target_condition'].values
    # Measure the accuracy by summing a coefficient for each suggestion, decreasing with the rank
    accuracy = 0
    for rk, condition in enumerate(target_conditions):
        if condition == user_condition:
            accuracy += 0.5 / 2**rk

    return accuracy

# Get the unique target conditions
unique_conditions = activities['target_condition'].unique()

# Initialize a dictionary to store the MSE for each condition
mse_by_condition = {condition: 0 for condition in unique_conditions}
accuracy_by_condition = {condition: 0 for condition in unique_conditions}

# Iterate over each condition
for condition in unique_conditions:
    # Get the user ids with the target condition
    user_ids = users[users['condition_id'] == condition]['user_id'].values
    # Calculate the MSE for the users with the target condition
    mse_by_condition[condition] = mean_squared_error(user_item_matrix_filled.loc[user_ids].values.flatten(), all_predictions[user_ids].flatten())
    # Calculate the accuracy for the users with the target condition
    for user_id in user_ids:
        accuracy_by_condition[condition] += custom_accuracy(user_id, all_predictions[user_id])
    # Normalize the accuracy
    accuracy_by_condition[condition] /= len(user_ids)

print("MSE by condition:", mse_by_condition)
print("Accuracy by condition:", accuracy_by_condition)

# Measure end time and CPU usage
cpu_usage_end = monitor_cpu()
memory_usage_end = monitor_memory()
end_time = time.time()

# Calculate total execution time and CPU usage
execution_time = end_time - start_time
cpu_usage = cpu_usage_end - cpu_usage_start

print(f"Total execution time: {execution_time} seconds")
print(f"CPU usage: {cpu_usage}%")
print(f"Memory usage: {memory_usage_end - memory_usage_start}%")

# Plotting performance metrics
plt.figure(figsize=(14, 6))

# CPU usage plot
plt.subplot(1, 3, 1)
plt.plot(cpu_usages, label='CPU Usage (%)')
plt.xlabel('Iterations')
plt.ylabel('CPU Usage (%)')
plt.title('CPU Usage Over Time')
plt.legend()

# Memory usage plot
plt.subplot(1, 3, 2)
plt.plot(memory_usages, label='Memory Usage (%)')
plt.xlabel('Iterations')
plt.ylabel('Memory Usage (%)')
plt.title('Memory Usage Over Time')
plt.legend()

# Execution time plot
plt.subplot(1, 3, 3)
plt.plot(execution_times, label='Execution Time (s)')
plt.xlabel('Iterations')
plt.ylabel('Execution Time (s)')
plt.title('Execution Time Over Time')
plt.legend()

plt.tight_layout()
plt.show()


0.6726731646460115
[[1.         0.32732684]
 [0.32732684 1.        ]]
Dimensions of user similarity matrix: (50, 50)
Number of users: 50
Number of items: 112
Length of one row of user item matrix: 112
Predicted preferences for user 0: [0.40096788 0.43504847 0.42062394 0.39735163 0.41409913 0.4671293
 0.42844352 0.41648886 0.39159219 0.42277709 0.42354304 0.42995743
 0.42693368 0.43964985 0.42974662 0.42299322 0.39058852 0.34748638
 0.43711889 0.43033181 0.42298422 0.42565464 0.40483332 0.43674444
 0.40960037 0.42499831 0.36133339 0.4324046  0.41301778 0.41346579
 0.7168039  0.41919052 0.44109384 0.42837728 0.39342592 0.42184039
 0.4044023  0.39695641 0.71507495 0.39861614 0.43212591 0.42253577
 0.43192204 0.27663036 0.43204495 0.4174741  0.41921652 0.43287905
 0.34128456 0.42230405 0.42478648 0.40094284 0.41753082 0.44234542
 0.45959583 0.42663837 0.40725242 0.44506997 0.41972025 0.42498317
 0.4430424  0.42543413 0.41573771 0.40884764 0.42384457 0.4120095
 0.43537013 0.42709261 0.41443